# UCB Algorithm

In [19]:
import math
import numpy as np
import csv

# Class BanditAlgorithm: Initialization

In [20]:
class BanditAlgorithm:
    def __init__(self, name):
        self.name = name
        self.results = []

    def add_result(self, timestep, iteration, total_reward, suboptimal_arms, regret, zeros_count, ones_count):
        self.results.append((timestep, iteration, total_reward, suboptimal_arms, regret, zeros_count, ones_count))

    def save_results_to_csv(self, filename):
        self.results.sort(key=lambda x: (x[1], x[0]))
        with open(filename, mode='w', newline='') as file:
            writer = csv.writer(file)
            writer.writerow(['Timestep', 'Iteration', 'Total Reward', 'Suboptimal Arms', 'Regret', 'Zeros Count', 'Ones Count'])
            for result in self.results:
                timestep = result[0]
                iteration = result[1]
                zeros_counts = sum(result[5])  # Summiere alle Nullen
                ones_counts = sum(result[6])   # Summiere alle Einsen
                writer.writerow([timestep, iteration, *result[2:5], zeros_counts, ones_counts])


    def calculate_average_results(self):
        time_steps = sorted(set(result[0] for result in self.results))
        avg_results = []
        for timestep in time_steps:
            total_reward_sum = 0
            suboptimal_arms_sum = 0
            regret_sum = 0
            zeros_count_sum = 0
            ones_count_sum = 0
            count = 0
            for result in self.results:
                if result[0] == timestep:
                    total_reward_sum += result[2]
                    suboptimal_arms_sum += result[3]
                    regret_sum += result[4]
                    zeros_count_sum += sum(result[5])  # Summiere alle Nullen in der Liste
                    ones_count_sum += sum(result[6])   # Summiere alle Einsen in der Liste
                    count += 1
            avg_total_reward = total_reward_sum / count if count > 0 else 0
            avg_suboptimal_arms = suboptimal_arms_sum / count if count > 0 else 0
            avg_regret = regret_sum / count if count > 0 else 0
            avg_zeros_count = zeros_count_sum / count if count > 0 else 0
            avg_ones_count = ones_count_sum / count if count > 0 else 0
            avg_results.append((timestep, avg_total_reward, avg_suboptimal_arms, avg_regret, avg_zeros_count, avg_ones_count))
        return avg_results


# Class UCB1

In [21]:
class UCB1:
    def __init__(self):
        self.counts = []
        self.values = []
        self.zeros_counts = []
        self.ones_counts = []
        
    def initialize(self, n_arms):
        self.counts = [0] * n_arms
        self.values = [0.0] * n_arms
        self.zeros_counts = [0] * n_arms
        self.ones_counts = [0] * n_arms
    
    def select_arm(self):
        n_arms = len(self.counts)
        for arm in range(n_arms):
            if self.counts[arm] == 0:
                return arm
        
        total_counts = sum(self.counts)
        ucb_values = [0.0] * n_arms
        
        for arm in range(n_arms):
            bonus = math.sqrt((2 * math.log(total_counts)) / self.counts[arm])
            ucb_values[arm] = self.values[arm] + bonus
        
        return ucb_values.index(max(ucb_values))
    
    def update(self, chosen_arm, reward):
        self.counts[chosen_arm] += 1
        n = self.counts[chosen_arm]
        value = self.values[chosen_arm]
        new_value = ((n - 1) / n) * value + (1 / n) * reward
        self.values[chosen_arm] = new_value
        if reward == 0:
            self.zeros_counts[chosen_arm] += 1
        else:
            self.ones_counts[chosen_arm] += 1

### Definition UCB

In [22]:
def UCB_simulation(algorithm, arm_means, total_steps):
    num_arms = len(arm_means)
    ucb = UCB1()
    ucb.initialize(num_arms)
    optimal_arm = np.argmax(arm_means)
    regret = np.zeros(total_steps)
    suboptimal_arms_count = 0
    total_regret = 0
    total_reward = 0

    for t in range(total_steps):
        chosen_arm = ucb.select_arm()
        reward = np.random.binomial(1, arm_means[chosen_arm])
        total_reward += reward
        ucb.update(chosen_arm, reward)
        regret[t] = arm_means[optimal_arm] - arm_means[chosen_arm]
        total_regret += regret[t]
        if chosen_arm != optimal_arm:
            suboptimal_arms_count += 1

    total_regret = round(total_regret, 1)

    return total_reward, suboptimal_arms_count, total_regret, ucb.zeros_counts, ucb.ones_counts


### Run Simulation Function

In [23]:
def run_simulation(algorithm, parameters, arm_means):
    for iteration in range(1, 101):  # Iteriere über 100 Durchläufe   
        for param in parameters:
            total_reward, suboptimal_arms_count, total_regret, zeros_count, ones_count = UCB_simulation(algorithm, arm_means, param)
            algorithm.add_result(param, iteration, total_reward, suboptimal_arms_count, total_regret, zeros_count, ones_count)


### UCB for different time horizons

In [24]:
# Beispiel-Parameter für die Zeit-Horizonte
time_horizons = [2, 3, 100, 200, 2000, 10000, 20000, 40000, 60000, 80000, 100000]

# Beispiel-Algorithmen
algorithms = [
    BanditAlgorithm("UCB"),
]

# Beispiel-Mittelwerte der Arme
arm_means = np.array([0.9, 0.8])  # Beispiel für die Mittelwerte der Arme

# Simulation durchführen und Ergebnisse speichern
for algorithm in algorithms:
    run_simulation(algorithm, time_horizons, arm_means)
    algorithm.save_results_to_csv(algorithm.name + '_results.csv')
    avg_results = algorithm.calculate_average_results()
    with open(algorithm.name + '_average_results.csv', mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['Timestep', 'Average Total Reward', 'Average Suboptimal Arms', 'Average Regret', 'Average Zeros Count', 'Average Ones Count'])
        for result in avg_results:
            writer.writerow(result)

hier wird deutlich, was bei nah beieinanderliegenden rewards passiert--> auch suboptimale Arme werden häufiger gezogen